In [1]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../utils')
from split_dataset import split_dataframe_by_keys
from dataset_loader import TidySequentialDataCSVLoader
%matplotlib inline

## Load the vitals and demographics

In [2]:
data_dir = '/cluster/tufts/hugheslab/datasets/eicu_v2.0/standardized_data_v2/'
ts_csv = os.path.join(data_dir, 'features_per_tstep.csv.gz')
outcomes_csv = os.path.join(data_dir, 'outcomes_per_seq.csv')

ts_df = pd.read_csv(ts_csv)
outcomes_df = pd.read_csv(outcomes_csv)

# keep only outcomes for which any data is observed
outcomes_df = outcomes_df[outcomes_df.icustay_id.isin(ts_df.icustay_id.unique())].copy().reset_index(drop=True)

In [3]:
ts_df

,icustay_id,hours_in,temperature,sao2,heartrate,respiration,systemicsystolic,systemicdiastolic,ALT (SGPT),AST (SGOT),...,Total CO2,fibrinogen,CRP,phosphate,direct bilirubin,troponin - T,subject_id,hadm_id,age,gender_is_male
0,141168,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,002-34851,128919,70,0
1,141168,1.0,NaN,93.000000,140.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,002-34851,128919,70,0
2,141168,2.0,NaN,NaN,136.833333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,002-34851,128919,70,0
3,141168,3.0,NaN,64.000000,134.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,002-34851,128919,70,0
4,141168,4.0,NaN,75.000000,132.833333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,002-34851,128919,70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261852,3353263,11.0,NaN,95.500000,79.583333,16.916667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,035-22638,2743110,35,0
4261853,3353263,12.0,NaN,93.750000,84.666667,16.916667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,035-22638,2743110,35,0
4261854,3353263,13.0,NaN,94.083333,84.000000,14.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,035-22638,2743110,35,0
4261855,3353263,14.0,NaN,94.750000,86.833333,15.750000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,035-22638,2743110,35,0


In [4]:
outcomes_df

,subject_id,hadm_id,icustay_id,mort,los_icu,hospitalid,unittype
0,002-34851,128919,141168,1,2.497222,59,Med-Surg ICU
1,002-33870,128927,141178,0,0.005556,60,Med-Surg ICU
2,002-33870,128927,141179,0,1.418056,60,Med-Surg ICU
3,002-5276,128941,141194,0,3.342361,73,CTICU
4,002-37665,128943,141196,0,1.015972,67,Med-Surg ICU
...,...,...,...,...,...,...,...
197048,035-16382,2743084,3353235,0,0.742361,458,Cardiac ICU
197049,035-751,2743086,3353237,0,0.881250,458,MICU
197050,035-5166,2743099,3353251,0,11.290972,458,Cardiac ICU
197051,035-19511,2743102,3353254,0,0.299306,459,Med-Surg ICU


## Remove stays less than 30 hours

In [6]:
min_stay_hrs = 30
min_stay_days = min_stay_hrs/24.0
keep_inds = outcomes_df['los_icu']>=min_stay_days
outcomes_df = outcomes_df.loc[keep_inds, :].copy().reset_index(drop=True)
ts_df = ts_df.loc[ts_df['icustay_id'].isin(outcomes_df['icustay_id']), :].reset_index(drop=True)


stay_lengths = outcomes_df['los_icu'].values
n_stays = len(outcomes_df['icustay_id'].unique())
n_patients = len(outcomes_df['subject_id'].unique())
n_deaths = outcomes_df['mort'].sum()

print('Total stays : %d'%n_stays)
print('Total patients : %d'%n_patients)
print('Frac of stays resulting in death : %.3f'%(n_deaths/n_stays))
print('Frac of patients who die : %.3f'%(n_deaths/n_patients))

Total stays : 113981
Total patients : 91174
Frac of stays resulting in death : 0.055
Frac of patients who die : 0.069


## Get the range of measurements of all features

In [3]:
ts_df.columns

Index(['icustay_id', 'hours_in', 'temperature', 'sao2', 'heartrate',
       'respiration', 'systemicsystolic', 'systemicdiastolic', 'ALT (SGPT)',
       'AST (SGOT)', 'BUN', 'Hct', 'Hgb', 'MCH', 'PT', 'RBC', 'WBC x 1000',
       'albumin', 'anion gap', 'calcium', 'chloride', 'creatinine', 'glucose',
       'platelets x 1000', 'potassium', 'sodium', 'total bilirubin',
       'total protein', 'uric acid', 'magnesium', 'bedside glucose', 'lactate',
       'HCO3', 'pH', 'FiO2', 'Total CO2', 'fibrinogen', 'CRP', 'phosphate',
       'direct bilirubin', 'troponin - T', 'subject_id', 'hadm_id', 'age',
       'gender_is_male'],
      dtype='object')

In [4]:
ts_feature_cols = ['temperature', 'sao2', 'heartrate',
       'respiration', 'systemicsystolic', 'systemicdiastolic', 'ALT (SGPT)',
       'AST (SGOT)', 'BUN', 'Hct', 'Hgb', 'MCH', 'PT', 'RBC', 'WBC x 1000',
       'albumin', 'anion gap', 'calcium', 'chloride', 'creatinine', 'glucose',
       'platelets x 1000', 'potassium', 'sodium', 'total bilirubin',
       'total protein', 'uric acid', 'magnesium', 'bedside glucose', 'lactate',
       'HCO3', 'pH', 'FiO2', 'Total CO2', 'fibrinogen', 'CRP', 'phosphate',
       'direct bilirubin', 'troponin - T', 'age',
       'gender_is_male']

id_col = ['icustay_id']
id_cols = ['subject_id', 'hadm_id', 'icustay_id']
time_col = ['hours_in']

feature_cols = ts_feature_cols
features_df = ts_df[id_cols+time_col+feature_cols].copy()

for lab in feature_cols:
    features_df[lab] = pd.to_numeric(features_df[lab], errors='coerce')

In [5]:
len(ts_feature_cols)

41

In [6]:
feats_summary_df = pd.DataFrame()
for lab in feature_cols:
    curr_lab_series = features_df[lab]
#     feats_summary_df.loc[lab, 'min'] = curr_lab_series.min()
#     feats_summary_df.loc[lab, 'max'] = curr_lab_series.max()
    feats_summary_df.loc[lab, '10%'] = np.nanpercentile(curr_lab_series, 10)
    feats_summary_df.loc[lab, '90%'] = np.nanpercentile(curr_lab_series, 90)
    feats_summary_df.loc[lab, 'median'] = curr_lab_series.median()


lab_counts_per_stay_df = features_df.groupby(id_col).count()
labs_missing_rate_entire_stay_dict = dict()
for lab in feature_cols:
    labs_missing_rate_entire_stay_dict[lab] = ((lab_counts_per_stay_df[lab]==0).sum())/lab_counts_per_stay_df.shape[0]
labs_missing_rate_entire_stay_series = pd.Series(labs_missing_rate_entire_stay_dict)

feats_summary_df.loc[:,'missing_rate'] = labs_missing_rate_entire_stay_series
feats_summary_df = feats_summary_df[['10%', 'median', '90%', 'missing_rate']]
feats_summary_df.to_csv('ts_feats_summary.csv')
feats_summary_df.round(2)

,10%,median,90%,missing_rate
temperature,35.80,37.18,38.21,0.91
sao2,93.25,97.17,100.00,0.04
heartrate,62.42,82.83,108.92,0.03
respiration,13.42,18.67,26.67,0.10
systemicsystolic,94.50,118.25,150.40,0.78
systemicdiastolic,44.25,57.45,75.42,0.78
ALT (SGPT),12.00,28.00,163.00,0.62
AST (SGOT),14.00,33.00,242.00,0.61
BUN,8.00,19.00,55.00,0.14
Hct,23.70,31.90,41.00,0.16


In [7]:
feats_summary_df.to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &         10\\% &      median &         90\\% &  missing\\_rate \\\\\n\\midrule\ntemperature       &   35.804167 &   37.183333 &   38.208333 &      0.914480 \\\\\nsao2              &   93.250000 &   97.166667 &  100.000000 &      0.041902 \\\\\nheartrate         &   62.416667 &   82.833333 &  108.916667 &      0.027302 \\\\\nrespiration       &   13.416667 &   18.666667 &   26.666667 &      0.103886 \\\\\nsystemicsystolic  &   94.500000 &  118.250000 &  150.400000 &      0.784089 \\\\\nsystemicdiastolic &   44.250000 &   57.454545 &   75.416667 &      0.784119 \\\\\nALT (SGPT)        &   12.000000 &   28.000000 &  163.000000 &      0.617184 \\\\\nAST (SGOT)        &   14.000000 &   33.000000 &  242.000000 &      0.611957 \\\\\nBUN               &    8.000000 &   19.000000 &   55.000000 &      0.141495 \\\\\nHct               &   23.700000 &   31.900000 &   41.000000 &      0.160064 \\\\\nHgb               &    7.800000 &   10.500000 &   13.800000

## Split in train/valid/test

In [12]:
random_state=41

x_train_df, x_test_df = split_dataframe_by_keys(
    features_df, cols_to_group=id_cols, size=0.2, random_state=random_state)

x_train_df, x_valid_df = split_dataframe_by_keys(
        x_train_df, cols_to_group=id_cols, size=0.2, random_state=random_state) 

y_train_df, y_test_df = split_dataframe_by_keys(
    outcomes_df, cols_to_group=id_cols, size=0.2, random_state=random_state)

y_train_df, y_valid_df = split_dataframe_by_keys(
        y_train_df, cols_to_group=id_cols, size=0.2, random_state=random_state) 


In [13]:
y_train_df

,subject_id,hadm_id,icustay_id,mort,los_icu,hospitalid,unittype
0,002-34851,128919,141168,1,2.497222,59,Med-Surg ICU
1,002-33870,128927,141179,0,1.418056,60,Med-Surg ICU
2,002-5276,128941,141194,0,3.342361,73,CTICU
3,002-23234,128948,141203,0,1.297917,66,Med-Surg ICU
5,002-23372,128982,141244,0,2.663194,73,CTICU
...,...,...,...,...,...,...,...
113975,035-18808,2743055,3353200,0,4.502083,458,Cardiac ICU
113976,035-18808,2743055,3353201,0,4.045139,458,Cardiac ICU
113978,035-2734,2743066,3353216,0,2.145833,458,CTICU
113979,035-14713,2743075,3353226,1,7.995139,458,Cardiac ICU


## Get the train valid test set stats and save data

In [15]:
for split, y_df, x_df in [('train', y_train_df, x_train_df),
                   ('valid', y_valid_df, x_valid_df),
                   ('test', y_test_df, x_test_df)]:

    stay_lengths = y_df['los_icu'].values
    n_stays = len(y_df['icustay_id'].unique())
    n_patients = len(y_df['subject_id'].unique())
    n_deaths = y_df['mort'].sum()

    print('Total stays : %d'%n_stays)
    print('Total patients : %d'%n_patients)
    print('Frac of stays resulting in death : %.3f'%(n_deaths/n_stays))
    print('Frac of patients who die : %.3f'%(n_deaths/n_patients))
    
    
    save_dir = '/cluster/tufts/hugheslab/datasets/eicu_v2.0/mortality_prediction/'
    for min_los in [3, 5, 7]:
        inds = stay_lengths>=min_los
        frac_above_min_los = len(stay_lengths[inds])/n_stays
        print('Frac stays > %d days in %s : %.3f'%(min_los, split, frac_above_min_los))
        y_df['los_geq_%s_days'%min_los] = (stay_lengths>=min_los)*1
    
    
    x_df.to_csv(os.path.join(save_dir, 'x_%s.csv'%split))   
    y_df.to_csv(os.path.join(save_dir, 'y_%s.csv'%split))  

Total stays : 72947
Total patients : 62322
Frac of stays resulting in death : 0.055
Frac of patients who die : 0.065
Frac stays > 3 days in train : 0.433
Frac stays > 5 days in train : 0.218
Frac stays > 7 days in train : 0.131
Total stays : 18237
Total patients : 17429
Frac of stays resulting in death : 0.056
Frac of patients who die : 0.058
Frac stays > 3 days in valid : 0.434
Frac stays > 5 days in valid : 0.217
Frac stays > 7 days in valid : 0.131
Total stays : 22797
Total patients : 21587
Frac of stays resulting in death : 0.055
Frac of patients who die : 0.058
Frac stays > 3 days in test : 0.431
Frac stays > 5 days in test : 0.219
Frac stays > 7 days in test : 0.131


## Save the fully supervised train/valid/test splits as numpy arrays for binary classification

In [17]:
## Convert from dataframe to numpy array of NxTxD
train_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_train_df,
    y_csv_path=y_train_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

valid_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_valid_df,
    y_csv_path=y_valid_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

test_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_test_df,
    y_csv_path=y_test_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

train_x_NTD, y_train = train_vitals.get_batch_data(batch_id=0)
valid_x_NTD, y_valid = valid_vitals.get_batch_data(batch_id=0)
test_x_NTD, y_test = test_vitals.get_batch_data(batch_id=0)

N_tr = len(train_x_NTD)
N_va = len(valid_x_NTD)
N_te = len(test_x_NTD)

print('positive label fraction in train : %.3f'%(y_train.sum()/len(y_train)))
print('positive label fraction in valid : %.3f'%(y_valid.sum()/len(y_valid)))
print('positive label fraction in test : %.3f'%(y_test.sum()/len(y_test)))

## save the data

# save the data to the respective folder
print('Saving data to %s'%save_dir)
np.save(os.path.join(save_dir, 'X_train.npy'), train_x_NTD)
np.save(os.path.join(save_dir, 'y_train.npy'), y_train)
print('Done saving train..')
np.save(os.path.join(save_dir, 'X_valid.npy'), valid_x_NTD)
np.save(os.path.join(save_dir, 'y_valid.npy'), y_valid)
print('Done saving valid..')
np.save(os.path.join(save_dir, 'X_test.npy'), test_x_NTD)
np.save(os.path.join(save_dir, 'y_test.npy'), y_test)
print('Done saving test..')


positive label fraction in train : 0.055
positive label fraction in valid : 0.056
positive label fraction in test : 0.055
Saving data to /cluster/tufts/hugheslab/datasets/eicu_v2.0/mortality_prediction/
Done saving train..
Done saving valid..
Done saving test..


## Save binary classification data based on % of labeled sequences

In [25]:
state_id = 41

## Convert from dataframe to numpy array of NxTxD
train_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_train_df,
    y_csv_path=y_train_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

valid_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_valid_df,
    y_csv_path=y_valid_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

test_vitals = TidySequentialDataCSVLoader(
    x_csv_path=x_test_df,
    y_csv_path=y_test_df,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort",
    y_label_type='per_sequence'
)

train_x_NTD, y_train = train_vitals.get_batch_data(batch_id=0)
valid_x_NTD, y_valid = valid_vitals.get_batch_data(batch_id=0)
test_x_NTD, y_test = test_vitals.get_batch_data(batch_id=0)

for ii, perc_labelled in enumerate([1.2, 3.7, 11.1, 33.3, 100]):#3.7, 11.1, 33.3, 100
    curr_save_dir = os.path.join(save_dir, 'percentage_labelled_sequences=%s'%perc_labelled)

    print('---------------------------------------------------------------------------')
    print('CREATING TRAIN/VALID/TEST SPLITS FOR %.3f PERCENT OF SEQUENCES LABELLED'%perc_labelled)
    print('---------------------------------------------------------------------------')
    y_train_ss = y_train.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_tr = int((1-(perc_labelled)/100)*N_tr)
    unlabelled_inds_tr = rnd_state.permutation(N_tr)[:n_unlabelled_tr]
    y_train_ss = y_train_ss.astype(np.float32)
    y_train_ss[unlabelled_inds_tr] = np.nan  
    if perc_labelled!=100:
        print('Excluded inds train: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_tr[0],
                                                              unlabelled_inds_tr[1],
                                                              unlabelled_inds_tr[2],
                                                              unlabelled_inds_tr[-3],
                                                              unlabelled_inds_tr[-2],
                                                              unlabelled_inds_tr[-1]))

    y_valid_ss = y_valid.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_va = int((1-(perc_labelled)/100)*N_va)
    unlabelled_inds_va = rnd_state.permutation(N_va)[:n_unlabelled_va]
    y_valid_ss = y_valid_ss.astype(np.float32)
    y_valid_ss[unlabelled_inds_va] = np.nan 
    if perc_labelled!=100:
        print('Excluded inds valid: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_va[0],
                                                          unlabelled_inds_va[1],
                                                          unlabelled_inds_va[2],
                                                          unlabelled_inds_va[-3],
                                                          unlabelled_inds_va[-2],
                                                          unlabelled_inds_va[-1]))

    y_test_ss = y_test.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_te = int((1-(perc_labelled)/100)*N_te)
    unlabelled_inds_te = rnd_state.permutation(N_te)[:n_unlabelled_te]
    y_test_ss = y_test_ss.astype(np.float32)
    y_test_ss[unlabelled_inds_te] = np.nan
    if perc_labelled!=100:
        print('Excluded inds test: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_te[0],
                                                          unlabelled_inds_te[1],
                                                          unlabelled_inds_te[2],
                                                          unlabelled_inds_te[-3],
                                                          unlabelled_inds_te[-2],
                                                          unlabelled_inds_te[-1]))

    # Check whether the specified path exists or not
    isExist = os.path.exists(curr_save_dir)

    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(curr_save_dir)

    # save the data to the respective folder
    print('Saving data to %s'%curr_save_dir)
    np.save(os.path.join(curr_save_dir, 'X_train.npy'), train_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_train.npy'), y_train_ss)
    print('Done saving train..')
    np.save(os.path.join(curr_save_dir, 'X_valid.npy'), valid_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_valid.npy'), y_valid_ss)
    print('Done saving valid..')
    np.save(os.path.join(curr_save_dir, 'X_test.npy'), test_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_test.npy'), y_test_ss)
    print('Done saving test..')


    print('---------------------------------------------------------------------------')
    for split, y in [('train', y_train_ss),
                    ('valid', y_valid_ss),
                    ('test', y_test_ss)]:
        frac_pos_labels = np.nansum(y)/(~np.isnan(y)).sum()
        print('fraction positive labels in %s set with %.3f percent of sequences labelled : %.4f'%(split,
                                                                                                   perc_labelled,
                                                                                                   frac_pos_labels))
    print('---------------------------------------------------------------------------')
    

---------------------------------------------------------------------------
CREATING TRAIN/VALID/TEST SPLITS FOR 1.200 PERCENT OF SEQUENCES LABELLED
---------------------------------------------------------------------------
Excluded inds train: 10817, 67247, 10519 ... 3509, 64906, 43242
Excluded inds valid: 1775, 6047, 858 ... 11398, 9140, 18073
Excluded inds test: 10289, 17460, 22795 ... 22403, 5127, 16104
Saving data to /cluster/tufts/hugheslab/datasets/eicu_v2.0/mortality_prediction/percentage_labelled_sequences=1.2
Done saving train..
Done saving valid..
Done saving test..
---------------------------------------------------------------------------
fraction positive labels in train set with 1.200 percent of sequences labelled : 0.0628
fraction positive labels in valid set with 1.200 percent of sequences labelled : 0.0502
fraction positive labels in test set with 1.200 percent of sequences labelled : 0.0511
---------------------------------------------------------------------------


In [18]:
patient_df = pd.read_csv('/cluster/tufts/hugheslab/datasets/eicu_v2.0/v20210518/patient.csv.gz')
patient_df['mort'] = (patient_df['unitdischargestatus']=='Expired')*1

In [22]:
patient_df['mort'].sum()/len(patient_df)

0.05430177388118033

In [24]:
len(ts_df['icustay_id'].unique())

113981